In [1]:
import math
import numpy as np
import pandas as pd
import catboost as cbt
import xgboost as xgb
import lightgbm as lgb
from sklearn.pipeline import Pipeline
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from catboost import CatBoostClassifier
from lightgbm import LGBMRegressor
from xgboost import XGBRegressor
from catboost import CatBoostRegressor
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, roc_auc_score,log_loss,precision_score, recall_score, f1_score 
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import KFold,StratifiedKFold,GridSearchCV
import matplotlib.pyplot as plt
from datetime import datetime,timedelta
import warnings
import os
import gc
import matplotlib.pyplot as plt
warnings.filterwarnings('ignore')
pd.options.display.max_columns = None
pd.options.display.max_rows = None

In [2]:
train = pd.read_csv(r'C:\Users\12239\Desktop\CFF\离散制造过程中典型工件的质量符合率预测\train_set\first_round_training_data - 副本.csv')
test = pd.read_csv(r'C:\Users\12239\Desktop\CFF\离散制造过程中典型工件的质量符合率预测\test_set\first_round_testing_data - 副本.csv')
submit = pd.read_csv(r'C:\Users\12239\Desktop\CFF\离散制造过程中典型工件的质量符合率预测\submit_example.csv')

In [3]:
quality_map = {'Excellent': 0, 'Good': 1, 'Pass': 2, 'Fail': 3}
train['label'] = train['Quality_label'].map(quality_map)
btype = 'lgb'
k = 5

In [4]:
parameter_name = ['Parameter{0}'.format(i) for i in range(5,11)]
attribute_name = ['Attribute{0}'.format(i) for i in range(4,9)]
X_train_para = train[parameter_name]
y_train_attr = train[attribute_name]
X_test_para = test[parameter_name]
y_train = train['label']
train_attr = np.zeros((X_train_para.shape[0],5))
test_attr = np.zeros((X_test_para.shape[0],5))
i = 0

for attr in attribute_name:
    kfold = KFold(n_splits=k, shuffle=True, random_state=2019)
    for train_index, test_index in kfold.split(X_train_para,y_train_attr):
        train_x, test_x, train_y, test_y = X_train_para.iloc[train_index], X_train_para.iloc[test_index], y_train_attr[attr].iloc[train_index], y_train_attr[attr].iloc[test_index]
        if btype == 'cbt':
            pipeline = Pipeline([('attr_model', CatBoostRegressor(random_state=2019))])
        elif btype == 'lgb':
            pipeline = Pipeline([('attr_model', LGBMRegressor(boosting_type="gbdt",random_state=2019))])
        elif btype == 'xgb':
            pipeline = Pipeline([('attr_model', XGBRegressor(random_state=2019))])

        n_estimators = range(500,1200,100)
        max_depth = [4,6,8,10]
        reg_alpha=[5,7,9,11,13]
        reg_lambda=[3,4,5,6]
        learning_rate=[0.01,0.03,0.05]
        random_grid = {
                        'attr_model__n_estimators': n_estimators,
                        'attr_model__max_depth': max_depth,
                        'attr_model__reg_alpha':reg_alpha,
                        'attr_model__reg_lambda':reg_lambda,
                        'attr_model__learning_rate':learning_rate
                       } 
        grid_search = GridSearchCV(pipeline,random_grid, n_jobs=-1, verbose=1, scoring=None)
        grid_search.fit(train_x,train_y)
        print('最佳效果：%0.3f' % grid_search.best_score_)
        best_parameters = grid_search.best_estimator_.get_params()
        print('最优参数：')
        for param_name in sorted(random_grid.keys()):
            print('\t%s: %r' % (param_name, best_parameters[param_name]))
        train_attr[test_index,i] += grid_search.predict(test_x)
        test_attr[:,i] += grid_search.predict(X_test_para)/k
    i += 1
print('mae',np.sum(np.absolute(train_attr - y_train_attr.values)/480))

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.


Fitting 3 folds for each of 1 candidates, totalling 3 fits


[Parallel(n_jobs=-1)]: Done   3 out of   3 | elapsed:    1.8s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.


最佳效果：-0.188
最优参数：
	attr_model__n_estimators: 1
Fitting 3 folds for each of 1 candidates, totalling 3 fits


[Parallel(n_jobs=-1)]: Done   3 out of   3 | elapsed:    1.3s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.


...

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   3 out of   3 | elapsed:    0.0s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   3 out of   3 | elapsed:    0.0s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   3 out of   3 | elapsed:    0.0s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   3 out of   3 | elapsed:    0.0s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   3 out of   3 | elapsed:    0.0s finished


最佳效果：-0.002
最优参数：
	attr_model__n_estimators: 1
Fitting 3 folds for each of 1 candidates, totalling 3 fits
最佳效果：0.009
最优参数：
	attr_model__n_estimators: 1
Fitting 3 folds for each of 1 candidates, totalling 3 fits
最佳效果：-0.867
最优参数：
	attr_model__n_estimators: 1
Fitting 3 folds for each of 1 candidates, totalling 3 fits
最佳效果：-1.173
最优参数：
	attr_model__n_estimators: 1
Fitting 3 folds for each of 1 candidates, totalling 3 fits
最佳效果：-2.771
最优参数：
	attr_model__n_estimators: 1
Fitting 3 folds for each of 1 candidates, totalling 3 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   3 out of   3 | elapsed:    0.0s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   3 out of   3 | elapsed:    0.0s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.


最佳效果：-0.704
最优参数：
	attr_model__n_estimators: 1
Fitting 3 folds for each of 1 candidates, totalling 3 fits
最佳效果：-0.099
最优参数：
	attr_model__n_estimators: 1
mae 44358570.085512206


[Parallel(n_jobs=-1)]: Done   3 out of   3 | elapsed:    0.0s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   3 out of   3 | elapsed:    0.0s finished


In [6]:
train[attribute_name] = train_attr
X_train = train[parameter_name+attribute_name]
y_train = train['label']
i = 0
for attr in attribute_name:
    test[attr] = test_attr[:,i]
    i += 1
X_test = test[parameter_name + attribute_name]
print(X_train.shape,X_test.shape)
oof1 = np.zeros((X_train.shape[0],4))
prediction1 = np.zeros((X_test.shape[0],4))
seeds = [x*x for x in range(10,20,2)]#设置随机种子
num_model_seed = len(seeds)
#params = {'max_depth':range(2, 7), 'n_estimators':range(100, 1100, 200), 'learning_rate':[0.01]}

for model_seed in range(num_model_seed):
    print(model_seed + 1)
    oof1_model = np.zeros((X_train.shape[0],4))
    prediction1_model=np.zeros((X_test.shape[0],4))
    skf = StratifiedKFold(n_splits=k, random_state=seeds[model_seed], shuffle=True)#构造交叉验证集，进行按比例取样（splits表示测试集和验证集为4：1）
    for index, (train_index, test_index) in enumerate(skf.split(X_train, y_train)):
        print(index)
        train_x, test_x, train_y, test_y = X_train.iloc[train_index], X_train.iloc[test_index], y_train.iloc[train_index], y_train.iloc[test_index]
        gc.collect()#进行垃圾内存回收
        model1 = cbt.CatBoostClassifier(iterations=1500,learning_rate=0.01,max_depth=6,verbose = 500,
                                           leaf_estimation_method='Newton',early_stopping_rounds=10000,
                                           task_type='GPU',loss_function='MultiClass')
        model1.fit(train_x, train_y ,eval_set=(train_x, train_y))
        oof1_model[test_index] = oof1_model[test_index]+model1.predict_proba(test_x) #预测验证集上某一个工件lebal为0到3的各自概率
        prediction1_model = prediction1_model+model1.predict_proba(X_test)/k #预测测试集上某一个工件lebal为0到3的各自概率（除10是因为splits=10，故要进行10次轮换交叉验证）
    oof1 = oof1+oof1_model / num_model_seed
    prediction1 =prediction1+prediction1_model / num_model_seed

    print('logloss',log_loss(pd.get_dummies(y_train).values, oof1_model))
    print('ac',accuracy_score(y_train, np.argmax(oof1_model,axis=1)))
    print('mae',1/(1 + np.sum(np.absolute(np.eye(4)[y_train] - oof1_model))/480))

print('logloss',log_loss(pd.get_dummies(y_train).values, oof1))
print('ac',accuracy_score(y_train, np.argmax(oof1,axis=1)))
print('mae',1/(1 + np.sum(np.absolute(np.eye(4)[y_train] - oof1))/480))
beta=2
p1=precision_score(y_train,np.argmax(oof1,axis=1),average='micro')
r1=recall_score(y_train,np.argmax(oof1,axis=1),average='micro')
p2=precision_score(y_train,np.argmax(oof1,axis=1),average='macro')
r2=recall_score(y_train,np.argmax(oof1,axis=1),average='macro')
print('micro precision',p1)
print('micro recall',r1)
print('macro precision',p2)
print('macro recall',r2)
print('micro F2-Score',(1+beta*beta)*p1*r1/(beta*beta*p1+r1))
print('macro F2-Score',(1+beta*beta)*p2*r2/(beta*beta*p2+r2))

(6000, 11) (6000, 11)
1
0
0:	learn: -1.3810173	test: -1.3810173	best: -1.3810173 (0)	total: 15.9ms	remaining: 143ms
9:	learn: -1.3410769	test: -1.3410768	best: -1.3410768 (9)	total: 153ms	remaining: 0us
bestTest = -1.341076767
bestIteration = 9
Shrink model to first 10 iterations.
1
0:	learn: -1.3809371	test: -1.3809372	best: -1.3809372 (0)	total: 22.8ms	remaining: 205ms
9:	learn: -1.3410162	test: -1.3410162	best: -1.3410162 (9)	total: 143ms	remaining: 0us
bestTest = -1.341016215
bestIteration = 9
Shrink model to first 10 iterations.
2
0:	learn: -1.3808888	test: -1.3808888	best: -1.3808888 (0)	total: 15ms	remaining: 135ms
9:	learn: -1.3407111	test: -1.3407110	best: -1.3407110 (9)	total: 167ms	remaining: 0us
bestTest = -1.340710981
bestIteration = 9
Shrink model to first 10 iterations.
3
0:	learn: -1.3807652	test: -1.3807654	best: -1.3807654 (0)	total: 17.7ms	remaining: 160ms
9:	learn: -1.3398076	test: -1.3398076	best: -1.3398076 (9)	total: 180ms	remaining: 0us
bestTest = -1.339807645
b

In [8]:
train[attribute_name] = train_attr
X_train = train[parameter_name+attribute_name]
y_train = train['label']
i = 0
for attr in attribute_name:
    test[attr] = test_attr[:,i]
    i += 1
X_test = test[parameter_name + attribute_name]
print(X_train.shape,X_test.shape)
oof2 = np.zeros((X_train.shape[0],4))
prediction2 = np.zeros((X_test.shape[0],4))
seeds = [x*x for x in range(10,20,2)]#设置随机种子
num_model_seed = len(seeds)
#params = {'max_depth':range(2, 7), 'n_estimators':range(100, 1100, 200), 'learning_rate':[0.01]}

for model_seed in range(num_model_seed):
    print(model_seed + 1)
    oof2_model = np.zeros((X_train.shape[0],4))
    prediction2_model=np.zeros((X_test.shape[0],4))
    skf = StratifiedKFold(n_splits=k, random_state=seeds[model_seed], shuffle=True)#构造交叉验证集，进行按比例取样（splits表示测试集和验证集为4：1）
    for index, (train_index, test_index) in enumerate(skf.split(X_train, y_train)):
        print(index)
        train_x, test_x, train_y, test_y = X_train.iloc[train_index], X_train.iloc[test_index], y_train.iloc[train_index], y_train.iloc[test_index]
        gc.collect()#进行垃圾内存回收
        model2 = XGBClassifier(
            silent=True ,
            learning_rate= 0.05, # 学习率
            min_child_weight=1,
            # 这个参数默认是 1，是每个叶子里面 h 的和至少是多少，对正负样本不均衡时的 0-1 分类而言
            #，假设 h 在 0.01 附近，min_child_weight 为 1 意味着叶子节点中最少需要包含 100 个样本。
            #这个参数非常影响结果，控制叶子节点中二阶导的和的最小值，该参数值越小，越容易 overfitting。
            max_depth=7, # 构建树的深度，越大越容易过拟合
            gamma=0.1,  # 树的叶子节点上作进一步分区所需的最小损失减少,越大越保守，一般0.1、0.2这样子。
            subsample=1, # 随机采样训练样本 训练实例的子采样比
            max_delta_step=0,#最大增量步长，我们允许每个树的权重估计。
            colsample_bytree=1, # 生成树时进行的列采样
            reg_lambda=1,  # 控制模型复杂度的权重值的L2正则化项参数，参数越大，模型越不容易过拟合。
            #reg_alpha=0, # L1 正则项参数
            scale_pos_weight=1, #如果取值大于0的话，在类别样本不平衡的情况下有助于快速收敛。平衡正负权重
            objective= 'multi:softmax', #多分类的问题 指定学习任务和相应的学习目标
            num_class=4, # 类别数，多分类与 multisoftmax 并用
            n_estimators=1800, #树的个数
            seed=2019, #随机种子
            eval_metric= 'mlogloss',)
        model2.fit(train_x, train_y,eval_metric='mlogloss')
        oof2_model[test_index] = oof2_model[test_index]+model2.predict_proba(test_x) #预测验证集上某一个工件lebal为0到3的各自概率
        prediction2_model = prediction2_model+model2.predict_proba(X_test)/k #预测测试集上某一个工件lebal为0到3的各自概率（除10是因为splits=10，故要进行10次轮换交叉验证）
    oof2 = oof2+oof2_model / num_model_seed
    prediction2 =prediction2+prediction2_model / num_model_seed

    print('logloss',log_loss(pd.get_dummies(y_train).values, oof2_model))
    print('ac',accuracy_score(y_train, np.argmax(oof2_model,axis=1)))
    print('mae',1/(1 + np.sum(np.absolute(np.eye(4)[y_train] - oof2_model))/480))

print('logloss',log_loss(pd.get_dummies(y_train).values, oof2))
print('ac',accuracy_score(y_train, np.argmax(oof2,axis=1)))
print('mae',1/(1 + np.sum(np.absolute(np.eye(4)[y_train] - oof2))/480))
beta=2
p1=precision_score(y_train,np.argmax(oof2,axis=1),average='micro')
r1=recall_score(y_train,np.argmax(oof2,axis=1),average='micro')
p2=precision_score(y_train,np.argmax(oof2,axis=1),average='macro')
r2=recall_score(y_train,np.argmax(oof2,axis=1),average='macro')
print('micro precision',p1)
print('micro recall',r1)
print('macro precision',p2)
print('macro recall',r2)
print('micro F2-Score',(1+beta*beta)*p1*r1/(beta*beta*p1+r1))
print('macro F2-Score',(1+beta*beta)*p2*r2/(beta*beta*p2+r2))

(6000, 11) (6000, 11)
1
0
1
2
3
4
logloss 1.243252484658414
ac 0.5153333333333333
mae 0.053806622071430624
2
0
1
2
3
4
logloss 1.2426384055605595
ac 0.5136666666666667
mae 0.05382582867836232
3
0
1
2
3
4
logloss 1.2420835262121512
ac 0.5168333333333334
mae 0.05383540646487394
4
0
1
2
3
4
logloss 1.2449834433817697
ac 0.5131666666666667
mae 0.05378109897024472
5
0
1
2
3
4
logloss 1.2441056435461844
ac 0.517
mae 0.053792503892009846
logloss 1.2416356234234318
ac 0.5255
mae 0.053808284454395186
micro precision 0.5255
micro recall 0.5255
macro precision 0.5005579051674008
macro recall 0.4676025897291758
micro F2-Score 0.5255
macro F2-Score 0.4738418712078171


In [9]:
train[attribute_name] = train_attr
X_train = train[parameter_name+attribute_name]
y_train = train['label']
i = 0
for attr in attribute_name:
    test[attr] = test_attr[:,i]
    i += 1
X_test = test[parameter_name + attribute_name]
print(X_train.shape,X_test.shape)
oof3 = np.zeros((X_train.shape[0],4))
prediction3 = np.zeros((X_test.shape[0],4))
seeds = [x*x for x in range(10,20,2)]#设置随机种子
num_model_seed = len(seeds)
#params = {'max_depth':range(2, 7), 'n_estimators':range(100, 1100, 200), 'learning_rate':[0.01]}

for model_seed in range(num_model_seed):
    print(model_seed + 1)
    oof3_model = np.zeros((X_train.shape[0],4))
    prediction3_model=np.zeros((X_test.shape[0],4))
    skf = StratifiedKFold(n_splits=k, random_state=seeds[model_seed], shuffle=True)#构造交叉验证集，进行按比例取样（splits表示测试集和验证集为4：1）
    for index, (train_index, test_index) in enumerate(skf.split(X_train, y_train)):
        print(index)
        train_x, test_x, train_y, test_y = X_train.iloc[train_index], X_train.iloc[test_index], y_train.iloc[train_index], y_train.iloc[test_index]
        gc.collect()#进行垃圾内存回收
        model3=LGBMClassifier(boosting_type="gbdt",num_leaves=23,reg_alpha=10, reg_lambda=5,max_depth=7,
                                 learning_rate=0.05, n_estimators=2000,colsample_bytree=0.7, subsample_freq=1,
                                 objective='multiclass', silent=True,subsample=0.7,min_child_samples=5,)
        model3.fit(train_x, train_y, eval_set=(test_x, test_y),early_stopping_rounds=1500, verbose=500,)#verbose=False   
        oof3_model[test_index] = oof3_model[test_index]+model3.predict_proba(test_x) #预测验证集上某一个工件lebal为0到3的各自概率
        prediction3_model = prediction3_model+model3.predict_proba(X_test)/k #预测测试集上某一个工件lebal为0到3的各自概率（除10是因为splits=10，故要进行10次轮换交叉验证）
    oof3 = oof3+oof3_model / num_model_seed
    prediction3 =prediction3+prediction3_model / num_model_seed

    print('logloss',log_loss(pd.get_dummies(y_train).values, oof3_model))
    print('ac',accuracy_score(y_train, np.argmax(oof3_model,axis=1)))
    print('mae',1/(1 + np.sum(np.absolute(np.eye(4)[y_train] - oof3_model))/480))

print('logloss',log_loss(pd.get_dummies(y_train).values, oof3))
print('ac',accuracy_score(y_train, np.argmax(oof3,axis=1)))
print('mae',1/(1 + np.sum(np.absolute(np.eye(4)[y_train] - oof3))/480))
beta=2
p1=precision_score(y_train,np.argmax(oof3,axis=1),average='micro')
r1=recall_score(y_train,np.argmax(oof3,axis=1),average='micro')
p2=precision_score(y_train,np.argmax(oof3,axis=1),average='macro')
r2=recall_score(y_train,np.argmax(oof3,axis=1),average='macro')
print('micro precision',p1)
print('micro recall',r1)
print('macro precision',p2)
print('macro recall',r2)
print('micro F2-Score',(1+beta*beta)*p1*r1/(beta*beta*p1+r1))
print('macro F2-Score',(1+beta*beta)*p2*r2/(beta*beta*p2+r2))

(6000, 11) (6000, 11)
1
0
Training until validation scores don't improve for 1000 rounds.
Did not meet early stopping. Best iteration is:
[10]	valid_0's multi_logloss: 1.23634
1
Training until validation scores don't improve for 1000 rounds.
Did not meet early stopping. Best iteration is:
[10]	valid_0's multi_logloss: 1.23791
2
Training until validation scores don't improve for 1000 rounds.
Did not meet early stopping. Best iteration is:
[10]	valid_0's multi_logloss: 1.23594
3
Training until validation scores don't improve for 1000 rounds.
Did not meet early stopping. Best iteration is:
[10]	valid_0's multi_logloss: 1.2424
4
Training until validation scores don't improve for 1000 rounds.
Did not meet early stopping. Best iteration is:
[10]	valid_0's multi_logloss: 1.23828
logloss 1.2381736343132588
ac 0.4028333333333333
mae 0.05499846148654473
2
0
Training until validation scores don't improve for 1000 rounds.
Did not meet early stopping. Best iteration is:
[10]	valid_0's multi_logloss

In [10]:
boosttype = 'lgb'

In [15]:
new_train = np.zeros((6000,12))
new_test = np.zeros((6000,12))
new_train[:,0:4] = oof1
new_train[:,4:8] = oof2
new_train[:,8:12] = oof3
new_test[:,0:4] = prediction1
new_test[:,4:8] = prediction2
new_test[:,8:12] = prediction3
if boosttype == 'cbt':
    pipeline = Pipeline([('attr_model', CatBoostClassifier(random_state=2019))])
elif boosttype == 'lgb':
    pipeline = Pipeline([('attr_model', LGBMClassifier(boosting_type="gbdt",random_state=2019))])
elif boosttype == 'xgb':
    pipeline = Pipeline([('attr_model', XGBClassifier(random_state=2019))])
n_estimators = [1]#range(500,1200,100)
#max_depth = [4,6,8]
#reg_alpha=[5,7,9]
#reg_lambda=[4,5,6]
#learning_rate=[0.01,0.03,0.05]
random_grid = {
                'attr_model__n_estimators': n_estimators,
                #'attr_model__max_depth': max_depth,
                #'attr_model__reg_alpha':reg_alpha,
                #'attr_model__reg_lambda':reg_lambda,
                #'attr_model__learning_rate':learning_rate
                } 
grid_search = GridSearchCV(pipeline,random_grid, n_jobs=-1, verbose=1, scoring=None)
grid_search.fit(new_train,train['label'].values)
print('最佳效果：%0.3f' % grid_search.best_score_)
best_parameters = grid_search.best_estimator_.get_params()
print('最优参数：')
for param_name in sorted(random_grid.keys()):
    print('\t%s: %r' % (param_name, best_parameters[param_name]))
prediction = grid_search.predict_proba(new_test)
print('mae',np.sum(np.absolute(np.argmax(prediction,axis=1) - train['label'].values)/480))

Fitting 3 folds for each of 1 candidates, totalling 3 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.


最佳效果：0.403
最优参数：
	attr_model__n_estimators: 1
mae 9.770833333333334


[Parallel(n_jobs=-1)]: Done   3 out of   3 | elapsed:    1.1s finished


In [16]:
sub = test[['Group']]
prob_cols = [i for i in submit.columns if i not in ['Group']]
for i, f in enumerate(prob_cols):
    sub[f] = prediction[:, i]
for i in prob_cols:
    sub[i] = sub.groupby('Group')[i].transform('mean')
sub = sub.drop_duplicates()

In [17]:
submit = sub.copy()
submit2 = sub.copy()
submit3 = sub.copy()
ratio = ['Excellent ratio','Good ratio','Pass ratio','Fail ratio']
submit[ratio]=submit[ratio]*50
submit2[ratio]=(submit2[ratio]*50).astype('int')
submit3[ratio]=(submit[ratio]-submit2[ratio]).astype('float')
submit2 = submit2.values
submit3 = submit3.values

In [18]:
k = np.argmax(submit3[:,1:5],axis=1) + 1
for i in range(120): 
    if submit3[i,k[i]] > 0.5:
        submit2[i,k[i]] += 1
        submit3[i,k[i]] = 0
k = np.argmax(submit3[:,1:5],axis=1) + 1
for i in range(120):
    if submit3[i,k[i]] > 0.5:
        submit2[i,k[i]] += 1
        submit3[i,k[i]] = 0
k = np.argmax(submit3[:,1:5],axis=1) + 1
for i in range(120):    
    if sum(submit2[i,1:5]) < 50:
        if submit3[i,k[i]] > 0.5:
            submit2[i,k[i]] += 1
            submit3[i,k[i]] = 0
k = np.argmax(submit3[:,1:5],axis=1) + 1
for i in range(120):    
    if sum(submit2[i,1:5]) < 50:
        submit2[i,k[i]] += 1
        submit3[i,k[i]] = 0
submit[ratio] = submit2[:,1:5] / 50

In [19]:
sub.to_csv(r'C:\Users\12239\Desktop\CFF\离散制造过程中典型工件的质量符合率预测\submit_example.csv',index=False)